In [13]:
import os 
from dotenv import load_dotenv

load_dotenv()

try:
    import google.generativeai as genai
except ImportError:
    genai = None

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

model_name = "gemini-2.0-flash"
model = genai.GenerativeModel(
    model_name,
    system_instruction="You are a concise, helpful assistant."

)
print("Client configured. model:", model_name)

Client configured. model: gemini-2.0-flash


In [14]:
import streamlit as st

# page configuration
st.set_page_config(page_title="World War", page_icon="🇬🇧", layout="wide")


#sidebar configuration
with st.sidebar:
    st.title("Settings")
    st.caption("🔑 Provide your API key & tweak generation parameters")
    
    # API key input
    api_key = st.text_input("GEMINI_API_KEY", type="password", 
                           value=os.getenv("GEMINI_API_KEY", ""))
    
    # model selection
    model_name = st.selectbox("Model", 
                              ["gemini-2.0-flash"])

    #  System instruction for world war focus
    systen_instruction = st.text_area(
        "System instruction",
        value="You are the world war assistant. you only provide information about world war, history, and the causes of the war",
        height=80
    )

    # Generation paramenters
    st.subheader("Generation Parameters")
    temperature = st.slider("Temperature", 0.0, 2.0, 0.7, 0.1)
    top_p = st.slider("Top P", 0.0, 1.0, 0.9, 0.05)
    top_k = st.slider("Top K", 1, 100, 40, 1)
    max_output_tokens = st.slider("Max Output Tokens", 1, 8000, 100, 100)
    
    

2025-09-05 05:39:14.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.290 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.291 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.295 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.296 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.296 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:39:14.297 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [18]:
# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "chat" not in st.session_state:
    st.session_state.chat = None

# Cache the model to avoid recreating it
@st.cache_resource(show_spinner=False)
def get_model(api_key, model_name, system_instruction):
    if genai is None:
        raise ImportError("google_generativeai is not installed")
    if not api_key:
            raise ValueError("API key missing")
        
    genai.configure(api_key=api_key)
    return genai.GenerativeModel(model_name, system_instruction=system_instruction)
    

# Main app
st.title("World War Chat")
st.caption("📚 Welcome to the catalogue of world war history - let's explore together!")

#display chat history 
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])

# user  input
user_prompt = st.chat_input("ask me anything about world war")      

# process user input
if user_prompt:
    #add user message to history    st.session_state.messages.append({'role': 'user', 'content': user_prompt})
    with st.chat_messages("user"):
        st.markdown(user_prompt)

    # Get model
    try:
        model = get_model(api_key, model_name, system_instruction)
    except Exception as e:
        st.error(str(e))
        st.stop()
    
    #generate configuration
    gen_config = {
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "max_output_tokens": max_output_tokens,
        "response_mime_type": "text/plain",
    }

    # Intialize chat session if needed
    if st.session_state.chat is None:
        history = []
        for msg in st.session_state.chat_messages[::-1]:
            role = "user" if msg["role"] == "user" else "model" 
            history.append({"role": role, "parts": msg["content"]})      
        st.session_state.chat = model.start_chat(history=history)


    # Generate response  
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            try:
                response = st.session_state.chat.send_message(user_prompt, generation_conf=gen_config)
                answer = response.txt or '(No text responde)'
            except Exception as e:
                answer = f"error {e}"
            st.markdown(answer) 

    # Add assistant response to the history
    st.session_state.messages.append({"role": "assistant", "content": answer})

2025-09-05 05:40:16.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.348 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.348 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 05:40:16.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar